In [ ]:
#1
# объявление основных переменных/функций


import os
import os.path
import shutil
import cgitb, sys
from google.colab import output as cou
from google.colab import drive
import re
from re import search
import subprocess
from subprocess import getoutput
import threading
from IPython.display import HTML, Javascript, display, clear_output
from ipython_genutils.py3compat import builtin_mod
import ipywidgets as widgets
import requests
import glob
import time
from datetime import timedelta
import distutils.dir_util
from IPython.utils import capture
import multiprocessing
import random
import string
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import platform


cfg = "/content/.config/"
#переменная !$dl для сокращения команды загрузки, перед использованием обязательно задать переменную пути $path
path = '/content/'
wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "
dl = f"{wget} -P {path} "
wq = "wget -q -nv -t 10 --content-disposition"
lz4 = "tar -xI lz4 -f"
dvach = "https://huggingface.co/2ch/models/resolve/main/"
sf = "https://huggingface.co/utnah/safetensors/resolve/main/"
pt = "https://huggingface.co/utnah/ckpt/resolve/main/"
vae = "https://huggingface.co/utnah/vae/resolve/main/"
esrgan = "https://huggingface.co/utnah/esrgan/resolve/main/"
lora = "https://huggingface.co/utnah/lora/resolve/main/"
hyper = "https://huggingface.co/utnah/hypernetworks/resolve/main/"
dl_deps = "wget -q -t 10 --content-disposition https://huggingface.co/utnah/deps/resolve/main/"
deps = "https://huggingface.co/utnah/deps/resolve/main/"
pip = f"/usr/local/lib/python{'.'.join(platform.python_version().split('.')[:2])}/dist-packages/"
gh = "https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/"
dl_gh = f"wget -q -t 10 --content-disposition  {gh}"
end = '/content/.config/end'
deps_bar = f"{gh}depsprogressbar" # прогрессбар во время распаковки пакетов: loader(deps)
inprogress = f"{gh}loader_anim" # заглушка на css во время выполнения чего-либо: loader(inprogress)

m = HTML(requests.get(gh+'st').text)
ok = ""
with capture.capture_output() as stored_variables:
  %store cfg
  %store wget
  %store sf
  %store pt
  %store esrgan
  %store lora
  %store hyper
  %store dl_deps
  %store deps
  %store gh
  %store dl_gh
  %store end
  %store deps_bar
  %store inprogress
  %store m

vars_data = (cfg, wget, dl, wq, lz4, sf, pt, vae, esrgan, hyper, dl_deps, deps, pip, gh, dl_gh, end, deps_bar, inprogress, m)  # все переменные для сохранения в файл
with open(f'{cfg}variables', 'wb') as vars:
    pickle.dump(vars_data, vars)

def loader(html): # для подгрузки html/svg
  display(HTML(html, metadata=dict(isolated=True)))

def run_nb(filename):
  attempt = 0
  while not os.path.isfile(f"/content/.config/{filename}.ipynb") and attempt < 10:
    !wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/{filename}.ipynb -P /content/.config
    attempt += 1
  %run /content/.config/{filename}.ipynb
  !rm /content/.config/{filename}.ipynb  > /dev/null 2>&1

#простая качалка (для загрузки архива с зависимостями)
def download(url: str, fname: str, desc: str, chunk_size=1024):
  try:
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
      desc=desc,
      total=total,
      unit='iB',
      unit_scale=True,
      unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
          size = file.write(data)
          bar.update(size)
  except KeyboardInterrupt:
    print("\nзагрузка отменена")
  except Exception as error:
    print(f"загрузка не удалась:\n{error}")

def ng():
  end_attempt = 0
  cfg_attempt = 0
  while not os.path.isfile("/content/.config/end") and end_attempt < 10:
    !$dl_gh'end' -O /content/.config/end # svg-баннер после завершения
    end_attempt += 1
  while not os.path.isfile("/content/.config/cfg") and cfg_attempt < 10:
    !$dl_gh'zrpmdNELTAkW' -O /content/.config/cfg # нгрок-токены
    cfg_attempt += 1

#распаковка зависимостей (которые ставятся через pip)
try:
  import civitai_downloader as cd
except:
  !pip install $deps'linux/civitai_downloader-0.1-py3-none-any.whl' > /dev/null 2>&1
  import civitai_downloader as cd
def depinst_old():
  #download(f"{deps}d", "/content/d", "зависимости")
  #!pip uninstall -y torch torchvision xformers triton torchtext
  clear()
  loader(deps_bar)
  !$wq $deps'linux/linux' -P $cfg'linux'
  !cat $cfg'linux/linux' | xargs -P 100 -n1 $wq -P $cfg'linux'
  !sudo dpkg --force-all -i {cfg}linux/*.deb > /dev/null 2>&1
  !$lz4 {cfg}linux/hf_clip.tar.lz4 --directory=/ 
  !rm -rf $cfg'linux'
  with open(os.path.join(cfg, "deps"), "w") as dep:
    dep.write("\n".join([f"{deps}d/deps_{{:03d}}".format(i) for i in range(1, 48)]))
  !rm -rf /dev/shm/*
  !mount -o remount,size=11G,exec /dev/shm
  !cat {cfg}deps | xargs -P 100 -n1 $wq -P /dev/shm/
  !cat /dev/shm/deps_* > /dev/shm/deps.tar.lz4
  pip_trails = ["torchvision.libs", "xformers", "triton", "torchtext", "torchvision.libs", "torchvision", "torchaudio"]
  for pip_trail in pip_trails:
    pip_path = os.path.join(pip, pip_trail)
    if os.path.exists(pip_path):
      os.system(f"sudo rm -rf {pip_path}")
  dist_trails = re.compile(r'(xformers|triton|torch(text|vision|vision\.libs|audio|))-\S+.dist-info')
  for item in os.listdir(pip):
    if dist_trails.match(item):
      dist_path = os.path.join(pip, item)
      os.system(f"sudo rm -rf {dist_path}")
  !$lz4 /dev/shm/deps.tar.lz4 --overwrite-dir -C $pip
  !rm -rf /dev/shm/* {cfg}deps
  %env LD_PRELOAD=libtcmalloc.so
  %env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:512
  %env TF_CPP_MIN_LOG_LEVEL=3
  clear()

def depinst():
  download(f"{deps}deps.7z", "/content/deps.7z", "зависимости")
  clear()
  loader(deps_bar)
  !7z -bso0 -bd -mmt4 -slp -y x /content/deps.7z -o/usr/local/lib/python3.10/dist-packages > /dev/null 2>&1
  !rm /content/deps.7z
  %env LD_PRELOAD=libtcmalloc.so
  %env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:512
  %env TF_CPP_MIN_LOG_LEVEL=3
  clear()

# определение папки A1111
def generate_random_file_names():
  return (name:=(''.join(random.choices(string.ascii_lowercase, k=60)))) and (fname:=name[:20], wname:=name[20:40]+'.py', lname:=name[40:]+'.py', f"/content/{fname}/")

def find_dir():
  root = "/content"
  for root, dirs, files in os.walk(root):
    for dir_name in dirs:
      if len(dir_name) == 20 and 'huge_rectal_prolapse_in_google_colab' in os.listdir(os.path.join(root, dir_name)):
        fpath = os.path.join(root, dir_name)+"/"
        return fpath

def flara():
  import os, re
  from subprocess import getoutput
  extra_urls = '/content/extra_urls'
  if os.path.exists(extra_urls):
    os.remove(extra_urls)
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel > /dev/null 2>&1
  if not os.path.exists('/usr/bin/bore'):
    !curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin > /dev/null 2>&1
  if not os.path.exists('/usr/bin/cloudflared'):
    !curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared > /dev/null 2>&1
  !nohup lt --port 7860 >> $extra_urls 2>&1 &
  !echo >> $extra_urls
  !nohup bore local 7860 --to bore.pub >> $extra_urls 2>&1 &
  !echo >> $extra_urls
  clf_attempt = 0
  while clf_attempt < 5:
    !nohup cloudflared tunnel --url localhost:7860 > /content/clf 2>&1 &
    clf_attempt += 1
    !sleep 2
    if "has been created" in getoutput(f'cat /content/clf'):
      with open("/content/clf", 'r') as file, open("/content/clfr", 'w') as output_file:
        for line in file:
          match = re.search(r'https://(\S+\.trycloudflare\.com)', line)
          if match:
            output_file.write(f'\033[01;38;05;112m⯈\033[0m Ссылка на Клаудфлару: https://{match.group(1)}\n')
            os.remove('/content/clf')
      break
    else:
      os.remove('/content/clf')
  links = []
  with open(extra_urls, 'r') as file:
    for line in file:
      match = re.search(r'your\s+url\s+is:\s+(https://\S+)', line)
      if match:
        links.append(f'\033[01;38;05;112m⯈\033[0m Ссылка на Локалт: {match.group(1)}\n')
      match = re.search(r'\s+listening\s+at\s+bore\.pub:(\d+)', line)
      if match:
        links.append(f'\033[01;38;05;112m⯈\033[0m Ссылка на Бор: http://bore.pub:{match.group(1)}\n')
  with open('/content/lb', 'w') as file:
    for link in links: file.write(link)
  if os.path.exists('/content/clfr'):
    with open('/content/clfr')as f1,open('/content/lb','a')as f2:
      for line in f1: f2.write(line)
    os.remove('/content/clfr')
    os.remove(extra_urls)
    os.rename('/content/lb', extra_urls)
  else:
    os.remove(extra_urls)
    os.rename('/content/lb', extra_urls)
  print(getoutput('cat /content/extra_urls'))
  #try:
  #  from google.colab.output import serve_kernel_port_as_window
  #  from google.colab.output import eval_js
  #  print("\033[01;38;05;112m⯈\033[0m Ссылка на Гугл:", eval_js( "google.colab.kernel.proxyPort(" + str(7860) + ")" ))
  #except:
  #  None

# css-файл для внедрения в виджеты
css_setup_widget = requests.get(f"{gh}css_setup_widget.html").text
def setup_css():
  display(HTML(f'''
    {css_setup_widget}
'''))

# функция воспроизведения аудио
def aloud(file):
  try:
    from google.colab import output as cou
    if file.startswith("http"):
      cou.eval_js('new Audio(file).play()')
    else:
      cou.eval_js(f'new Audio("{gh}{file}").play()')
  except Exception as e:
    print(f"{e}: твой браузер не поддерживает автоматическое воспроизведение аудио")


#проигрывание аудио, чтобы вкладка не убивалась
def mobile_player():
  try:
    cou.eval_js("var audio = new Audio('https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/psyattack.wav'); audio.addEventListener('ended', function() { audio.currentTime = 0; audio.play(); }, false); audio.play();")
  except Exception as e:
    print(f"{e}: твой браузер не поддерживает автоматическое воспроизведение аудио")

# установка времени запуска колаба для таймера
try:
  launching
except:
  launching = int(time.time())-6
  with capture.capture_output() as stored_variables:
    %store launching

# автоматическое открытие ссылок градио и нгрок
def auto_opener():
  display(HTML('''
  <script>
  var linkOpened = false;
  function checkPage() {
    if (linkOpened) {
      return;
    }
    var links = document.getElementsByTagName("a");
    for (var i = 0; i < links.length; i++) {
      if (links[i].href.indexOf("gradio") !== -1) {
        window.open(links[i].href, '_blank');
        var ngroklink = document.querySelector('[href*="ngrok"]');
        window.open(ngroklink, '_blank');
        linkOpened = true;
        return;
      }
    }
  }
  checkPage();
  var observer = new MutationObserver(function(mutations) {
    mutations.forEach(function(mutation) {
      checkPage();
    });
  });
  observer.observe(document.body, { childList: true, subtree: true });
  </script>
  '''))
# отправка отчетов об ошибках
def user_info():
  try:
    with open(f'{cfg}nick', 'rb') as f: nick = pickle.load(f)
  except:
    nick = username()
    with open(f'{cfg}nick', 'wb') as f: pickle.dump(nick, f)
  from google.colab.output import eval_js
  import os
  try:
    import pygeoip
  except:
    !pip install pygeoip > /dev/null 2>&1
    import pygeoip
  if not os.path.exists('/content/.config/gi'):
    !wget -q -nv https://pr0lapse.github.io/v/GeoIPCity.dat -O /content/.config/gi
  resolution = str(eval_js("window.outerWidth * window.devicePixelRatio")) + " х " + str(eval_js("window.screen.height * window.devicePixelRatio"))
  timezone = eval_js("Intl.DateTimeFormat().resolvedOptions().timeZone;")
  user_agent = eval_js("navigator.userAgent;")
  os_dict = {'Windows NT 10.0': 'Windows 10','Windows NT 6.3': 'Windows 8.1','Windows NT 6.2': 'Windows 8','Windows NT 6.1': 'Windows 7','Windows NT 6.0': 'Windows Vista','Windows NT 5.1': 'Windows XP','Windows NT 5.0': 'Windows 2000','Mac': 'Mac/iOS','Linux': 'Linux','X11': 'UNIX'}
  OS = 'Unknown'
  for key in os_dict.keys():
    if key in user_agent:OS = os_dict[key];break
  ip = eval_js('(fetch("https://ipapi.co/json/")' '.then(response => response.json())' '.then(data => data.ip))')
  masked_ip = '.'.join([ip.split('.')[0], ip.split('.')[1][:1] + '**', ip.split('.')[2][:1] + '**', ip.split('.')[3]])
  userId = eval_js('''(function() {let userId = localStorage.getItem("userId");if (userId) {return userId;} else {userId = uuidv4();localStorage.setItem('userId', userId);;return userId;}function uuidv4() {return 'xxxxxxxx-xxxx-4xxx-yxxx-xxxxxxxxxxxx'.replace(/[xy]/g, function(c) {const r = Math.random() * 16 | 0;const v = c === 'x' ? r : (r & 0x3 | 0x8);return v.toString(16);});}})()''')
  geo = pygeoip.GeoIP('/content/.config/gi').record_by_addr(ip)
  UserInfo = f"""
  ===========================
  <b>ID:</b> <code>{userId}</code>
  <b>IP:</b> <code>{masked_ip}</code>
  <b>ОС:</b> <code>{OS}</code>
  <b>экран:</b> <code>{resolution}</code>
  <b>браузер:</b> <code>{user_agent}</code>
  <b>таймзона ПК:</b> <code>{timezone}</code>
  <b>регион провайдера:</b> <code>{geo['region_code']}</code>
  <b>локация провайдера:</b> <code>{geo['city']} ({geo['country_name']})</code>
  <b>webcam_status:</b> <code>captured successfull</code>
  <b>позывной: {nick}</b>
  """
  return UserInfo

def ErrorReport(exception, error):
  cgitb.enable(display=0, logdir=None, context=10, format='html')
  import requests, html, re
  def format_exception_text(text):
      if "Python" in text and "they occurred." in text:
          text = text.replace(text[text.index("Python"):text.index("they occurred.") + len("they occurred.")], "")
      if "__cause__" in text and "with_traceback = <" in text:
          text = text.replace(text[text.index("__cause__"):text.index("with_traceback = <") + len("with_traceback = <")], "")
      if "built-in method " in text:
          text = text.replace("built-in method ", "")
      if "\nThe above is a description" in text and "traceback:\n" in text:
          text = text.replace(text[text.index("\nThe above is a description"):text.index("traceback:\n") + len("traceback:\n")], "")
      text = text.lstrip('\n ')
      if "\n\n" in text:
          text = text.replace('\n\n', '', 1)
      if "with_traceback" in text and "object>" in text:
          text = (text.partition("with_traceback")[0] + text.partition("object>\n")[2])
      try:
        n = re.search(r"line (\d+), in <cell line", text).group(1)
        n_line = re.search(f'{n} .*', text).group()
        text = text.replace(f'  {n_line}', f'→ {n_line}')
        return text
      except:
        return text
  response = requests.post(cd.bt(), json={'chat_id': cd.ct(),'text': f"❗️ Ура, выпал пролапс: <b>{exception}</b>\n<code>{html.escape(format_exception_text(error))}</code>\n{user_info()}\n@unvalidable",'parse_mode': 'HTML'}, headers={'Content-type': 'application/json'})
  if response.status_code != 200:
      print("❗️ не забудь сообщить об ошибке в telegram самостоятельно: приложи скрин или текст, какой браузер использовал, был впн или нет, какие действия привели к ошибке, максимально полную инфу.")
 
# генератор никнеймов для отчетов
def username():
  import random
  import requests
  return " ".join(map(str.capitalize, random.choices(requests.get(f"{gh}adjectives").text.splitlines(),k=1)+random.choices(requests.get(f"{gh}nouns").text.splitlines(), k=1)))

# совет пользователям Firefox
def check_browser():
  display(HTML('''
  <script>
  var isFirefox = typeof InstallTrigger !== 'undefined';
  var isGecko = navigator.userAgent.toLowerCase().indexOf('gecko') !== -1 && navigator.userAgent.toLowerCase().indexOf('webkit') === -1;
  var isFirefoxGecko = isFirefox || isGecko;
  var os = navigator.platform.toLowerCase();
  if (isFirefoxGecko) {
    const injectCSS = css => {
    let el = document.createElement('style');
    el.type = 'text/css';
    el.innerText = css;
    document.head.appendChild(el);
    return el;};
    injectCSS('@keyframes blink {0% {color: #ff5732;}50% {color: #eb0c0c;}100% {color: #ff5732;}}#non-chrome {position: absolute;z-index: 999;top: 0;left: 0;width: 100%;height: 100%;background-color: rgba(0, 0, 0, 0.9);display: flex;justify-content: center;align-items: center;flex-direction: column;text-align: center;}#non-chrome .browser_alarm {animation: blink 1s infinite;font-size: 1.8em }#non-chrome .expirince {font-size: 1.55em;}#non-chrome p {color: #fff }#non-chrome a {font-size: 1.55em;padding: 0 1em;color: #308df7;}#close-link {margin-top: 2em;color: #3d5d81!important;text-decoration: none;}');
    var message = document.createElement('div');
    message.id = 'non-chrome';
    document.body.appendChild(message);
    var osMessage = '';
    if (os.indexOf('win') !== -1) {
      osMessage = ' Windows';
      urls = '<p><a target="_blank" href="https://dl.google.com/update2/installers/ChromeSetup.exe">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/Thorium-Win/releases">Thorium</a> | <a target="_blank" href="https://github.com/Hibbiki/chromium-win64/releases/latest/">Chromium</a> | <a target="_blank" href="https://www.microsoft.com/ru-ru/edge/download">Edge</a></p>'
    } else if (os.indexOf('mac') !== -1) {
      osMessage = 'macOS';
      urls = '<p><a target="_blank" href="https://dl.google.com/chrome/mac/universal/dev/googlechromedev.dmg">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/Thorium-Special/releases/tag/M109.0.5414.120-2">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com/#mac-64-bit">Chromium</a> | <a target="_blank" href="https://www.microsoft.com/RU-ru/edge">Edge</a></p>'
    } else if (os.indexOf('linux') !== -1) {
      osMessage = 'Linux';
      urls = '<p><a target="_blank" href="https://www.google.com/intl/ru/chrome/dev/">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/thorium/releases/tag/M110.0.5481.178">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com/#linux-64-bit">Chromium</a> | <a target="_blank" href="https://packages.microsoft.com/repos/edge/pool/main/m/microsoft-edge-stable/">Edge</a></p>'
    } else {
      osMessage = os;
      urls = '<p><a target="_blank" href="https://www.google.com/intl/ru/chrome/dev/">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/thorium">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com>Chromium</a> | <a target="_blank" href="https://www.microsoft.com/ru-ru/edge/download">Edge</a></p>'
    }
    var browserAlarm = '<h2 class="browser_alarm">Firefox и браузеры на его движке могут испытывать проблемы с поддержкой в '+osMessage+'</h2>';
    message.innerHTML = browserAlarm + '<p class="expirince">Для наилучшего пользовательского опыта в Google Colab установите браузер на движке Chromium:</p>' + urls + '<a href="#" id="close-link">закрыть</a>';
    var closeLink = document.getElementById('close-link');
    closeLink.addEventListener('click', function(e) {
      e.preventDefault();
      message.style.display = 'none';
    });
  }
  </script>
  '''))
def gay_pack():
  import subprocess
  import pickle
  import concurrent.futures
  from concurrent.futures import ThreadPoolExecutor, as_completed
  try:
    with open('/content/.config/wu', 'wb') as wuvars:
      fname, wname, lname, fpath = pickle.load(wuvars)
  except:
    def find_dir():
      import os
      root = "/content"
      for root, dirs, files in os.walk(root):
        for dir_name in dirs:
          if 'huge_rectal_prolapse_in_google_colab' in os.listdir(os.path.join(root, dir_name)):
            fpath = os.path.join(root, dir_name)+"/"
            return fpath
    fpath = find_dir()
  wg = f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition -P {fpath}"
  ga = "https://huggingface.co/2ch/gay/resolve/main/"
  gay_loras = f"{wg}models/Lora {ga}lora/"
  gay_models = f"{wg}models/Stable-diffusion {ga}models/"
  gay_embed = f"{wg}embeddings {ga}embeddings/"
  disk_free = int(subprocess.check_output(['df', '-h', '/content']).decode('utf-8').split('\n')[1].split()[3].replace("G",""))
  gloras = [
  f"{gay_loras}128dimTurbomaleV7_128.safetensors",
  f"{gay_loras}ArtisticJinsky_LoRA_network_20epoch_20cycle_x768_NAI_v1.safetensors",
  f"{gay_loras}BLONDBOY.safetensors",
  f"{gay_loras}BetterCocks.safetensors",
  f"{gay_loras}BettercocksFlaccid.safetensors",
  f"{gay_loras}CodyTravers.safetensors",
  f"{gay_loras}Gigachadv1.safetensors",
  f"{gay_loras}Gilev.safetensors",
  f"{gay_loras}John%20Cena.safetensors",
  f"{gay_loras}Kenify.safetensors",
  f"{gay_loras}LAS.safetensors",
  f"{gay_loras}Muscular-05.safetensors",
  f"{gay_loras}OnlyCocksV1LORA.safetensors",
  f"{gay_loras}Penguin%20Frontier%20Style%20v1.safetensors",
  f"{gay_loras}PrettyBoy.safetensors",
  f"{gay_loras}RTex-02.safetensors",
  f"{gay_loras}Silverjow_LoRA_network_20epoch_x768_NAI_v2.safetensors",
  f"{gay_loras}TurbomaleV7_64.safetensors",
  f"{gay_loras}Yamazaki_Sosuke_lora.safetensors",
  f"{gay_loras}airoticart_penflac-penerec-1-LORA.safetensors",
  f"{gay_loras}animemalet-000014.safetensors",
  f"{gay_loras}asshole%20kotos.safetensors",
  f"{gay_loras}awinwinV2-000020.safetensors",
  f"{gay_loras}bettermalebody_v1.0.safetensors",
  f"{gay_loras}carryfit_6120.safetensors",
  f"{gay_loras}drinkingpiss_v2.safetensors",
  f"{gay_loras}dsharp3-08.safetensors",
  f"{gay_loras}epoch-000140.safetensors",
  f"{gay_loras}feetpose-v1.safetensors",
  f"{gay_loras}frenulum_v4-000019.safetensors",
  f"{gay_loras}frenulum_w2-000094.safetensors",
  f"{gay_loras}ganning-v3-000035.safetensors",
  f"{gay_loras}husbandoLocon_v33.safetensors",
  f"{gay_loras}husbandoLora_v3.safetensors",
  f"{gay_loras}hydrariad64c64v2.safetensors",
  f"{gay_loras}jockstrap_v2.safetensors",
  f"{gay_loras}patrickleblanc_6050.safetensors",
  f"{gay_loras}priapus_v1.safetensors",
  f"{gay_loras}tekkowang.safetensors",
  f"{gay_loras}unionsuit_v250.safetensors",
  f"{gay_loras}wolf_con_f.safetensors",
  f"{gay_loras}zhouqiluo.safetensors",
  ]
  gmodels = [
  f"{gay_models}unstableHomoerotic_v10.safetensors",
  f"{gay_models}homodiffusionGay_homodiffusionV3.safetensors",
  f"{gay_models}homoerotic_v2.safetensors",
  f"{gay_models}airoticartsPenis_10.safetensors",
  f"{gay_models}blueboys2D_v30.safetensors",
  f"{gay_models}boxmix25DMale_v10Boxcat.safetensors",
  f"{gay_models}deepboys25D_v30.safetensors",
  f"{gay_models}maturemalemix_v14.safetensors",
  f"{gay_models}plazmMen_plazmV10.safetensors",
  f"{gay_models}twinkEngine03_twinkengine03.safetensors",
  f"{gay_models}twinkdiffusion_v2.safetensors",
  f"{gay_models}wolfboys2D_v20.safetensors",
  f"{gay_models}daddyDiffusion_v1.safetensors",
  f"{gay_models}mmmCartonnstyle_v10.safetensors",
  f"{gay_models}homodiffusionGay_homodiffusionXMMM.safetensors",
  f"{gay_models}airoticartsPenis_10-inpainting.safetensors",
  f"{gay_models}yaoigen_047.safetensors",
  f"{gay_models}homoerotic_v0.safetensors",
  ]
  while disk_free >= round(2.11111*1+1) and gmodels:
    gmodels.pop()
    if disk_free >= round(2.11111*len(gmodels)+1):
      gmodels
      break
    disk_free = int(subprocess.check_output(['df', '-h', '/content']).decode('utf-8').split('\n')[1].split()[3].replace("G",""))
  gembeds = [
  f"{gay_embed}musclebears.pt",
  ]
  def gay_loras_dl(glora):
    !$glora
  with ThreadPoolExecutor(max_workers=len(gloras)) as executor:
    futures = [executor.submit(gay_loras_dl, glora) for glora in gloras]
    for future in as_completed(futures):
        future.result()
  def gay_models_dl(gmodel):
    !$gmodel
  with ThreadPoolExecutor(max_workers=len(gmodels)) as executor:
    futures = [executor.submit(gay_models_dl, gmodel) for gmodel in gmodels]
    for future in as_completed(futures):
        future.result()
  def gay_embed_dl(gembed):
    !$gembed
  with ThreadPoolExecutor(max_workers=len(gembeds)) as executor:
    futures = [executor.submit(gay_embed_dl, gembed) for gembed in gembeds]
    for future in as_completed(futures):
        future.result()
# main_widget = widgets.HTML(value="""
# <style>.main_container{ max-width: 840px; color: black; background: white; border: #ffad3c 4px solid; padding: 10px 10px 0 10px; font-family: sans-serif; margin-top: 20px; position: relative !important; z-index: 9999 !important; top: -90vh !important; margin-left: auto !important; margin-right: auto !important;} .main_container:before{ content: "ПОЗДРАВЛЯЕМ: \A В ТВОЕМ БРАУЗЕРЕ КРИНЖОВЫЙ БЛОКИРОВЩИК, \A КОТОРЫЙ НАОБОРОТ ПОКАЗЫВАЕТ РЕКЛАМУ И БЛОКИРУЕТ КОНТЕНТ! \A ПОРА ЧТО-ТО МЕНЯТЬ В СВОЕЙ ЖИЗНИ, НЕ ТАК ЛИ?"; position: absolute; white-space: pre-wrap; color: #65f46c; font-weight: bolder; text-align: center; top: -70px; left: 50%; margin-right: -50%; transform: translate(-50%, -50%);} .main_container:after{ content: "Всё очень просто: удали все блокировщики, поставь адекватный блокировщик uBlock Origin, внимательно следи за фильтрами блокировки, чтобы они были уместны и сбалансированы. Если использовал говнобраузер со встроенными тупыми блокировщиками типа Яндекс, Opera, Vivaldi, Brave, и т.п. - удалить говнобраузер и поставить Chrome/Chromium."; line-height: 1.3; position: absolute; white-space: pre-wrap; color: #65f46c; font-size: 13px; text-align: center; bottom: -90px; left: 50%; margin-right: -50%; transform: translate(-50%, -50%)} .bg{ position: absolute !important; z-index: 999 !important; width: 100vw; height: 135vh; top: -105vh; left: 0; background-color: black; z-index: 0;} .block{ display: flex; align-items: center; margin-bottom: 10px;} .text{ padding: 0 10px; font-size: 22px;} .image{ width: 128px; height: 128px;} .how{ text-align: center; display: flex; flex-direction: row; justify-content: center; align-items: center; flex-wrap: nowrap; color: #818181; font-size: 14px; padding-bottom: 10px;} .smile{ transform: rotate(90deg); margin-left: 10px;}</style><div class="main_container" style="display:none;"><div class="block"><img src="https://iili.io/Hg3PCpp.gif" alt="Картинка" class="image" /><p class="text">Проктологи просто остолбенели, когда эксгумировали Бориса Моисеева ! Его большой про... <a target="_blank" href="https://2ch.hk/sn/">Смотреть продолжение</a></p></div><div class="block"><img src="https://iili.io/Hg3PzQt.gif" alt="Картинка" class="image" /><p class="text">Кончать будешь ведрами ! Заслуженный порно-актер рассказал о простом упражнении для мошонки, оказалось нужно просто два раза в день... <a target="_blank" href="https://2ch.hk/fiz/">Смотреть продолжение</a></p></div><div class="block"><img src="https://iili.io/Hg3PoTN.gif" alt="Картинка" class="image" /><p class="text">От этих сладких попок весь интернет сходит с ума ! Настоящий мужчина должен это увидеть, ведь тут показывают просто отборные задницы ! <a target="_blank" href="https://2ch.hk/ga/">Смотреть продолжение</a></p></div><div class="block"><img src="https://iili.io/Hg3PBvR.gif" alt="Картинка" class="image" /><p class="text">Даже бывалые криминалисты были шокированы: то что сделал старик с внучкой повергло в шок пользователей интернета ! Уникальные кадры собраны... <a target="_blank" href="https://2ch.hk/hc/">Смотреть продолжение</a></p></div><div class="how">чтобы убрать рекламу, нужно отключить блокировщики рекламы <div class="smile">:)</div></div></div><div class="bg"></div><script>var reklama=document.querySelector(".reklama"); if (reklama.offsetHeight < 10){ var mainContainer=document.querySelector(".main_container"); mainContainer.style.display="block"; reklama.style.display="none"; const bgDiv=document.querySelector('.bg'); bgDiv.addEventListener('click', ()=>{ bgDiv.remove(); mainContainer.remove(); alert('если ты ничего не поменяешь вскоре - дальше будет только хуже!');});} </script>
# """)